In [1]:
import pandas as pd
import numpy as np
import os
import math
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM, BatchNormalization
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler, StandardScaler
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
# dir_path = './' #local
dir_path = './drive/MyDrive/BAKA/' #colab

In [2]:
#uncomment in colab
from google.colab import drive
drive.mount('/content/drive')
# pd.read_csv(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', sep=' ', index_col=False )
# test = np.genfromtxt(dir_path + 'DataCollection' + '/' + '1' + '/' + '0.txt', delimiter=' ',dtype='float64')
df = pd.read_csv(dir_path + 'DataCollection' + '/' + '7' + '/' + '0.txt', header=None, sep=' ')
# df.drop(df.index[100:])
# filler = np.zeros((100-len(df), 31))
# df = df.append(pd.DataFrame(filler), ignore_index=True )
# df
if len(df.columns) > 31:
  df = df.drop(columns=[31])
df
# sp_df = np.array_split(df, 2)
# sp_df[0]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
0,170.415,171.583,170.530,171.003,174.445,154.652,163.519,166.115,165.641,162.487,1.253290,194.882,14.9770,28.96000,189.262,-35.603500,45.53200,185.628,-45.093400,61.10950,180.397,-34.183900,73.7031,181.117,-8.99203,48.142200,217.883,49.5742,3.75821,2.104970,4.13247
1,169.329,172.484,170.150,171.597,174.496,152.873,165.641,165.928,166.465,163.127,-8.616610,191.839,15.2395,17.57970,188.431,-38.200900,33.65670,182.574,-46.765500,50.02620,176.931,-35.898400,62.9153,177.678,-10.54940,37.379600,215.672,48.2805,3.47132,1.757100,3.07063
2,167.655,173.037,169.813,171.482,173.459,150.460,166.703,165.740,166.740,163.775,-19.081500,189.688,14.7336,5.56185,187.971,-40.991500,21.53430,180.185,-49.011700,38.13220,174.031,-38.143400,51.8784,174.688,-12.96820,25.849700,213.251,46.7521,2.55655,1.690020,2.64563
3,166.571,173.503,169.197,171.677,172.458,148.909,167.125,165.531,167.099,163.716,-31.304300,190.266,15.0975,-7.45626,187.463,-42.522700,8.86878,178.043,-49.996400,25.58150,171.274,-39.126900,39.5904,171.037,-13.77820,13.845000,211.619,45.9054,2.16560,1.736170,2.39077
4,165.696,175.323,170.447,172.439,173.044,147.501,169.382,166.810,168.132,165.104,-44.488500,191.750,15.3027,-19.92800,188.088,-44.296900,-3.93108,176.573,-51.103000,12.17790,169.156,-40.123200,26.3311,168.231,-14.56410,0.791638,209.579,45.8894,1.54479,1.682020,2.90702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,161.179,149.029,152.195,151.016,150.205,152.997,126.959,124.467,127.138,123.439,-30.090100,184.622,38.2253,-37.25150,196.216,0.788915,-25.96480,177.172,0.577421,-6.30855,169.466,1.548870,16.9119,170.591,10.74000,20.691400,219.560,52.5405,2.51276,0.449101,10.67120
96,160.792,151.716,152.281,150.662,149.034,151.989,126.807,125.764,127.363,122.859,-19.908700,181.773,35.9630,-26.55300,188.969,0.309116,-15.62690,172.253,-1.190490,4.36088,165.698,0.422513,26.8130,168.415,10.13660,29.739400,217.434,50.6622,3.39557,0.308685,8.73040
97,160.418,150.423,151.525,148.881,146.811,151.084,123.746,125.563,125.645,121.057,-9.673830,180.636,35.3971,-14.54250,184.884,1.792140,-4.11838,169.524,-1.451800,16.21090,164.016,1.369290,37.9906,167.859,11.25630,39.445900,216.438,50.7116,4.15535,0.403556,7.08729
98,160.405,148.238,151.163,147.639,144.736,150.975,120.556,124.997,123.932,119.012,0.594164,179.349,34.9790,-2.71361,183.277,3.301830,7.35355,168.698,-1.786510,27.97520,163.769,2.372690,49.2641,168.144,12.92220,49.179900,215.835,51.6498,4.51655,0.880348,5.78411


In [3]:
files = os.listdir(dir_path + 'DataCollection')
files

['0', '2', '3', '4', '1', '5', '6', '7']

In [4]:
x_train = [];
y_train = [];

x_test = [];
y_test = [];
for i in files:
    samples = os.listdir(dir_path + 'DataCollection' + '/' + i)
    num_tests = int(len(samples)/5);
    shuffle(samples, random_state = 0)
    for k in range(0, num_tests):
#         df = np.genfromtxt(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], delimiter=' ',dtype='float64')
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_test.append(df.to_numpy())
        y_test.append(int(i));
    
    for k in range(num_tests, len(samples)):
        df = pd.read_csv(dir_path + 'DataCollection' + '/' + i + '/' + samples[k], header=None, sep=' ')
        if len(df.columns) > 31:
          df = df.drop(columns=[31])
        if df.isnull().values.any():
          print(i, " ", samples[k])
        # df = df.drop(df.index[100:])
        # if len(df) < 100:
        #     filler = np.zeros((100-len(df), 31))
        #     df = df.append(pd.DataFrame(filler), ignore_index=True )
        x_train.append(df.to_numpy())
        y_train.append(int(i));
    
    print(len(samples), ' ', num_tests, ' ', len(samples)- num_tests)
# print(len(x_train))
x_train = np.array(x_train)
y_train = np.array(y_train);

x_test = np.array(x_test)
y_test = np.array(y_test)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)


302   60   242
512   102   410
302   60   242
410   82   328
602   120   482
422   84   338
201   40   161
201   40   161
x_train.shape:  (2364, 100, 31)
y_train.shiape:  (2364,)
x_test.shape:  (588, 100, 31)
y_test.shape:  (588,)


In [5]:
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)



x_train.shape:  (2364, 100, 31)
y_train.shiape:  (2364,)
x_test.shape:  (588, 100, 31)
y_test.shape:  (588,)


In [6]:
def scale_data(data, min_max_scaler):
    for i in range(len(data)):
        data[i] = min_max_scaler.transform(data[i])
    return data

In [7]:
min_max_scaler = MinMaxScaler(feature_range=(0,1))

num_instances, num_time_steps, num_features = x_train.shape
x_train = np.reshape(x_train, newshape=(-1, num_features))
x_train = min_max_scaler.fit_transform(x_train)
x_train = np.reshape(x_train, newshape=(num_instances, num_time_steps, num_features))

x_train, y_train = shuffle(x_train, y_train, random_state=0)

num_instances, num_time_steps, num_features = x_test.shape
x_test = np.reshape(x_test, newshape=(-1, num_features))
x_test = min_max_scaler.transform(x_test)
x_test = np.reshape(x_test, newshape=(num_instances, num_time_steps, num_features))

x_test, y_test = shuffle(x_test, y_test, random_state=0)


In [8]:
# x_train = np.delete(x_train, [0,1,2,3,4,5,6,7,8], 0)
# y_train = np.delete(y_train, [0,1,2,3,4,5,6,7,8], 0)
print("x_train.shape: ", x_train.shape)
print("y_train.shiape: ", y_train.shape)
print("x_test.shape: ", x_test.shape)
print("y_test.shape: ", y_test.shape)
# print(y_train)
# print("\n")
# print(y_test)

# y_train = y_train.astype('int')

# print(y_train)


x_train.shape:  (2364, 100, 31)
y_train.shiape:  (2364,)
x_test.shape:  (588, 100, 31)
y_test.shape:  (588,)


In [9]:
model = Sequential()
model.add(LSTM(units=100, input_shape=x_train.shape[1:], return_sequences=True,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100, return_sequences=True ,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(LSTM(100,dtype='float64'))
model.add(BatchNormalization())
model.add(Dropout(0.2))

# model.add(Dense(30, activation='softmax'))

model.add(Dense(8, activation='softmax',dtype='float64'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100, 100)          52800     
_________________________________________________________________
batch_normalization (BatchNo (None, 100, 100)          400       
_________________________________________________________________
dropout (Dropout)            (None, 100, 100)          0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 100)          80400     
_________________________________________________________________
batch_normalization_1 (Batch (None, 100, 100)          400       
_________________________________________________________________
dropout_1 (Dropout)          (None, 100, 100)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 100)          8

In [10]:
opt = tf.keras.optimizers.Adam(lr=0.0001, decay=1e-5)

checkpoint_filepath = dir_path + 'Checkpoints/'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy'],
)


gestures = model.fit(x = x_train,
            y = y_train,
            epochs=50,
            validation_split=0.1, #split 10% of the trainning set for the validation set,
            batch_size=24,
            callbacks=[model_checkpoint_callback],
            shuffle=True
         )

Epoch 1/50
89/89 [==============================] - 40s 64ms/step - loss: 1.3608 - accuracy: 0.5520 - val_loss: 1.9001 - val_accuracy: 0.5190
Epoch 2/50
89/89 [==============================] - 4s 47ms/step - loss: 0.3158 - accuracy: 0.9189 - val_loss: 1.3572 - val_accuracy: 0.6709
Epoch 3/50
89/89 [==============================] - 4s 47ms/step - loss: 0.2033 - accuracy: 0.9435 - val_loss: 0.7546 - val_accuracy: 0.7384
Epoch 4/50
89/89 [==============================] - 4s 47ms/step - loss: 0.1873 - accuracy: 0.9452 - val_loss: 0.4006 - val_accuracy: 0.8523
Epoch 5/50
89/89 [==============================] - 4s 47ms/step - loss: 0.1915 - accuracy: 0.9414 - val_loss: 0.2047 - val_accuracy: 0.9156
Epoch 6/50
89/89 [==============================] - 4s 47ms/step - loss: 0.1325 - accuracy: 0.9554 - val_loss: 0.0968 - val_accuracy: 0.9578
Epoch 7/50
89/89 [==============================] - 4s 47ms/step - loss: 0.1404 - accuracy: 0.9583 - val_loss: 0.4011 - val_accuracy: 0.8861
Epoch 8/50
8

In [11]:
# model.save(dir_path + 'Models/gestures.h5')

In [12]:
print("Evaluate on test data")
results = model.evaluate(x_test, y_test, batch_size=24)
print("test loss, test acc:", results)
model.save(dir_path + 'Models/gestures_simplified_dynamic', save_format='tf')

Evaluate on test data
25/25 [==============================] - 0s 15ms/step - loss: 0.0958 - accuracy: 0.9745
test loss, test acc: [0.09583073854446411, 0.9744898080825806]


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_simplified_dynamic/assets


INFO:tensorflow:Assets written to: ./drive/MyDrive/BAKA/Models/gestures_simplified_dynamic/assets


In [13]:
# new_model = load_model(dir_path + 'Models/gestures.h5')
# print("Evaluate on test data")
# results = new_model.evaluate(x_test, y_test, batch_size=80)
# print("test loss, test acc:", results)

In [16]:
min_max_scaler.data_max_

array([180.   , 179.999, 180.   , 179.996, 179.995, 179.999, 179.989,
       180.   , 179.991, 179.986, 579.577, 719.193, 570.106, 549.272,
       778.684, 578.49 , 533.127, 791.146, 551.114, 543.578, 783.643,
       520.983, 550.029, 762.745, 508.462, 519.313, 723.459, 526.573,
       173.127, 162.605, 164.575])

In [15]:
model.input

<KerasTensor: shape=(None, 100, 31) dtype=float64 (created by layer 'lstm_input')>